<a href="https://colab.research.google.com/github/NDVDS/YOLOv5-Cars-detecting/blob/main/Copy_of_CarsDetecting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!git clone https://github.com/NDVDS/YOLOv5-Cars-detecting.git
%cd /content/YOLOv5-Cars-detecting

In [ ]:
#Install requirements and load pretrained yolov5s model
!pip install -r https://raw.githubusercontent.com/ultralytics/yolov5/master/requirements.txt

import torch
model = torch.hub.load('ultralytics/yolov5', 'yolov5s', pretrained = True)

In [ ]:
#Modifying calsses and confidence parameters
model.classes = 2 #Cars only
model.conf = 0.73

In [ ]:
im = '/content/imageSample.jpg'

In [ ]:
#Model inference and printing results in json format
results = model(im)
bboxes = results.pandas().xyxy[0][['xmin','ymin','xmax','ymax']]
print(bboxes)

In [ ]:
results.show()

In [ ]:
#Saving JSON
bboxes.to_json('BBOXESPred.json',orient='index')

In [ ]:
#VIDEO:
#Extracting farme by frame foward to model input,
#writing results to new video.
import cv2
from tqdm import tqdm

vid_cap = cv2.VideoCapture('/content/videoInput.mp4')
frame_read, img = vid_cap.read()
      
fps = vid_cap.get(cv2.CAP_PROP_FPS) 
w = int(vid_cap.get(cv2.CAP_PROP_FRAME_WIDTH))
h = int(vid_cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
frame_count = int(vid_cap.get(cv2.CAP_PROP_FRAME_COUNT))
out = cv2.VideoWriter('/content/videoOutput.mp4', cv2.VideoWriter_fourcc(*'MP4V'), fps, (w,h))

while(frame_read):
  for i in tqdm(range(frame_count)):
    result = model(img)
    im = result.render()[0]
    out.write(im)
    frame_read, img = vid_cap.read()
    
out.release()
vid_cap.release()